In [ ]:
!pip install tensorflow==2.10.0 #tensorflow 2.10.0 <-- 2.12.0 (버전 다운그레이드 필요)

In [2]:
# Microsoft/DialoGPT-large(file size = 3GB) supports multiple languages

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

from transformers import AutoTokenizer, TFAutoModelForCausalLM
from timeit import default_timer

print_current_datetime()

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

print_current_datetime("Saving Microsoft DialoGPT Model")

tokenizer.save_pretrained("DialoGPT-large-by-Microsoft-Keras")
model.save_pretrained("DialoGPT-large-by-Microsoft-Keras")

print_current_datetime("Loading Microsoft DialoGPT Model")

tokenizer_loaded = tokenizer.from_pretrained("DialoGPT-large-by-Microsoft-Keras")
model_loaded = model.from_pretrained("DialoGPT-large-by-Microsoft-Keras")

print_current_datetime()

def generate_ouput(prompt=""):

    start = default_timer()

    input_ids = tokenizer_loaded.encode(prompt, return_tensors="tf")
    output_ids = model_loaded.generate(input_ids=input_ids,
                                       max_length=1024,
                                       temperature=0.7,
                                       top_p=0.9,
                                       do_sample=True,
                                       num_return_sequences=5, # The model will generate five different responses to the prompt.
                                       pad_token_id=tokenizer_loaded.eos_token_id)
    generated_text = tokenizer_loaded.decode(output_ids[0], skip_special_tokens=True)

    end = default_timer()

    # num_return_sequences=5, which means the model will generate 5 different responses to the prompt.
    # The below code loops through the generated responses and print them out with a response number.
    for i, return_sequence in enumerate(output_ids):
        print(f'Response {i+1}: {tokenizer_loaded.decode(return_sequence, skip_special_tokens=True)}')

    print("Time duration(in seconds):", end - start)
    return generated_text

# Let's chat for 10 lines
for step in range(10):
    prompt = input(">> User:")
    if prompt.lower() == "bye": break

    generated_text = generate_ouput(prompt)
    split_generated_text = generated_text.split(prompt)
    if len(split_generated_text) > 1:
        generated_text = split_generated_text[1]
    # Trim the sentences after the last period(.)
    text_to_remove = generated_text.split('.')[-1]
    generated_text = generated_text.replace(text_to_remove,'')

    print(">> GPT: {}".format( generated_text ))
    print_current_datetime()

 @ CDT(2023-04-25T14:22:00.786168)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Saving Microsoft DialoGPT Model @ CDT(2023-04-25T14:22:12.347576)
Loading Microsoft DialoGPT Model @ CDT(2023-04-25T14:22:38.448764)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at DialoGPT-large-by-Microsoft-Keras.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


 @ CDT(2023-04-25T14:22:46.148085)
>> User:Hi
Response 1: Hi
Response 2: Hi, I'm not sure if it's possible to change the name of the name of the name.
Response 3: Hi
Response 4: Hi, I think that's the first time this season, I've seen it, and it's a good thing.I'm glad you're having a good time
Response 5: Hi. I would like to know how to get a copy of the game. it's not really a game
Time duration(in seconds): 29.757516394005506
>> GPT: 
 @ CDT(2023-04-25T14:23:18.788136)
>> User:Do you speak Korean?
Response 1: Do you speak Korean?
Response 2: Do you speak Korean?
Response 3: Do you speak Korean?
Response 4: Do you speak Korean?
Response 5: Do you speak Korean?
Time duration(in seconds): 1.0472906050126767
>> GPT: 
 @ CDT(2023-04-25T14:23:32.307827)
>> User:굉장해요.
Response 1: 굉장해요.You've been waiting to use that one for a while now.
Response 2: 굉장해요.jpg.
Response 3: 굉장해요.gif
Response 4: 굉장해요.That's how you spell it.
Response 5: 굉장해요.He was very good at being a politician.
Time duration

In [1]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("DialoGPT-large-by-Microsoft-Keras")
model = TFAutoModelForCausalLM.from_pretrained("DialoGPT-large-by-Microsoft-Keras")

# Fine-tune the model with the below sentences
sentences = [
    "Hello, how are you?",
    "I'm doing well, thanks for asking.",
    "What have you been up to lately?",
    "Not much, just working on some projects.",
    "That sounds interesting. What kind of projects?",
    "Just some coding projects for work.",
    "Ah, I see. What kind of coding do you do?",
    "Mostly Python and Java.",
    "Cool, I've been meaning to learn Python.",
    "It's a great language. You should definitely give it a try!"
]

input_ids = tokenizer.encode("\n".join(sentences), return_tensors='tf')
outputs = model(input_ids)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=[loss, loss])

model.fit(input_ids, input_ids, epochs=10)

# Generate fine-tuned outputs
input_ids = tokenizer.encode("What is a great language?", return_tensors='tf')

finetuned_output = model.generate(input_ids=input_ids,
                   max_length=1024,
                   temperature=0.7,
                   top_p=0.9,
                   do_sample=True,
                   num_return_sequences=5, # The model will generate five different responses to the prompt.
                   pad_token_id=tokenizer.eos_token_id)

for i, return_sequence in enumerate(finetuned_output):
    print(f'Response {i+1}: {tokenizer.decode(return_sequence, skip_special_tokens=True)}')

2023-04-25 15:26:03.226837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 15:26:03.353421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 15:26:03.353441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-25 15:26:03.383047: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-25 15:26:03.934319: W tensorflow/stream_executor/pla

Epoch 1/10
1/1 [==============================] - 51s 51s/step - loss: 11.5285
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 9.0290
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 7.7521
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 6.5265
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 5.7008
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 5.1092
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 4.4682
Epoch 8/10
1/1 [==============================] - 3s 3s/step - loss: 3.7823
Epoch 9/10
1/1 [==============================] - 3s 3s/step - loss: 3.1973
Epoch 10/10
1/1 [==============================] - 3s 3s/step - loss: 2.6637
Response 1: What is a great language? Java, Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Java Ja